In [182]:
import random
import pandas as pd
import pickle
import re
import sys
from load_data import *

In [183]:
''' TEM(Typed Entity Marker) 후 각 entity의 start_idx, end_idx 재설정하는 한다. '''


def update_idx(dataset):
    update_sub_idx, update_obj_idx= [], []
    for sen,sub_idx,obj_idx in zip(dataset['sentence'],dataset['subject_idx'],dataset['object_idx']):
     
        # TEM(Typed Entity Marker) 후 subject_idx 재설정
        sub_start_idx = sen.find('@')
        sub_end_idx = sub_start_idx+(sub_idx[1]-sub_idx[0]+1)+6
        new_sub_i = [sub_start_idx, sub_end_idx]
        update_sub_idx.append(new_sub_i)
            
        # TEM(Typed Entity Marker) 후 object_idx 재설정    
        obj_start_idx = sen.find('#')
        obj_end_idx = obj_start_idx+(obj_idx[1]-obj_idx[0]+1)+6
        new_obj_i = [obj_start_idx, obj_end_idx]
        update_obj_idx.append(new_obj_i)
    
    out_sentence = pd.DataFrame({'id': dataset['id'], 'sentence' : dataset['sentence'], 'subject_entity': dataset['subject_entity'], 
                                       'object_entity': dataset['object_entity'],'subject_type': dataset['subject_type'],
                                       'object_type': dataset['object_type'], 'label': dataset['label'],
                                       'subject_idx': update_sub_idx, 'object_idx': update_obj_idx})
    return out_sentence




'''update_idx 통과 후 랜덤으로 0~1사이의 숫자를 뽑아 확률 p보다 작을 시 random으로 한 단어를 삭제해주는 함수'''
def random_delete(dataset, p):
    new_sentence= []
    for sen,sub_idx,obj_idx in zip(dataset['sentence'],dataset['subject_idx'],dataset['object_idx']):
        if random.random() <= p:

            sub_start_idx = sen.find('@')
            sub_len = sub_idx[1]-sub_idx[0]+1
            tmp_sub = sen[sub_start_idx:sub_start_idx+sub_len]
            
            obj_start_idx = sen.find('#')
            obj_len = obj_idx[1]-obj_idx[0]+1
            tmp_obj = sen[obj_start_idx:obj_start_idx+obj_len]
            
            # TEM한 sub_entity를 @로 바꿔준다
            sen=sen.replace(tmp_sub,'@')

            # TEM한 obj_entity를 #로 바꿔준다
            sen=sen.replace(tmp_obj,'#')
            is_delete = False

            '''
            위의 두 번의 치환 후 다음과 같은 결과가 나온다.
            Before
            〈Something〉는 #^PER^조지 해리슨#이 쓰고 @*ORG*비틀즈@가 1969년 앨범 《Abbey Road》에 담은 노래다.

            After
            〈Something〉는 #이 쓰고 @가 1969년 앨범 《Abbey Road》에 담은 노래다.
            '''

            words = sen.split()
            while is_delete == False:
                '''
                After된 문장의 길이가 9이다.
                0~8 사이의 임의의 정수를 뽑아 @, #이 아닌
                '''
                delete_idx = random.randint(0,len(words)-1)

                # 선택한 단어에 @ or #이 들어있지 않다면 삭제해준다.
                if '@' not in words[delete_idx] and '#' not in words[delete_idx]:
                    is_delete=True
                    del words[delete_idx]
                    sen=" ".join(words)
                    sen=sen.replace('@',tmp_sub)
                    sen=sen.replace('#',tmp_obj)
                    new_sentence.append(sen)

        else:
            new_sentence.append(sen)

    out_sentence = pd.DataFrame({'id': dataset['id'], 'sentence' : new_sentence, 'subject_entity': dataset['subject_entity'], 
                                       'object_entity': dataset['object_entity'],'subject_type': dataset['subject_type'],
                                       'object_type': dataset['object_type'], 'label': dataset['label'],
                                       'subject_idx': dataset['subject_idx'], 'object_idx': dataset['object_idx']})
    return out_sentence




'''update_idx 통과 후 랜덤으로 0~1사이의 숫자를 뽑아 확률 p보다 작을 시 두개의 단어를 swap해주는 함수'''
def random_swap(dataset, p):
    new_sentence= []
    for sen,sub_idx,obj_idx in zip(dataset['sentence'],dataset['subject_idx'],dataset['object_idx']):
        
        sub_start_idx = sen.find('@')
        sub_len = sub_idx[1]-sub_idx[0]+1
        tmp_sub = sen[sub_start_idx:sub_start_idx+sub_len]
            
        obj_start_idx = sen.find('#')
        obj_len = obj_idx[1]-obj_idx[0]+1
        tmp_obj = sen[obj_start_idx:obj_start_idx+obj_len]

        if random.random() <= p:
            sen = sen.replace(tmp_sub,"@")
            sen = sen.replace(tmp_obj,"#")
            
            words = sen.split()
            random_idx_1 = random.randint(0, len(words) - 1)
            random_idx_2 = random_idx_1
           
            while random_idx_2 == random_idx_1:
                random_idx_2 = random.randint(0, len(words) - 1)

            words[random_idx_1], words[random_idx_2] = words[random_idx_2], words[random_idx_1]
            sen =" ".join(words)
            sen = sen.replace('@',tmp_sub)
            sen = sen.replace('#',tmp_obj)
            new_sentence.append(sen)
        else:
            new_sentence.append(sen)

    out_sentence = pd.DataFrame({'id': dataset['id'], 'sentence' : new_sentence, 'subject_entity': dataset['subject_entity'], 
                                       'object_entity': dataset['object_entity'],'subject_type': dataset['subject_type'],
                                       'object_type': dataset['object_type'], 'label': dataset['label'],
                                       'subject_idx': dataset['subject_idx'], 'object_idx': dataset['object_idx']})
    return out_sentence

In [184]:
train_dataset = load_data('../dataset/train/train_no_dup.csv')
train_dataset

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,〈Something〉는 #^PER^조지 해리슨#이 쓰고 @*ORG*비틀즈@가 196...,비틀즈,조지 해리슨,ORG,PER,no_relation,"[24, 26]","[13, 18]"
1,1,호남이 기반인 바른미래당·#^ORG^대안신당#·@*ORG*민주평화당@이 우여곡절 끝...,민주평화당,대안신당,ORG,ORG,no_relation,"[19, 23]","[14, 17]"
2,2,K리그2에서 성적 1위를 달리고 있는 @*ORG*광주FC@는 지난 26일 #^ORG...,광주FC,한국프로축구연맹,ORG,ORG,org:member_of,"[21, 24]","[34, 41]"
3,3,균일가 생활용품점 (주)@*ORG*아성다이소@(대표 #^PER^박정부#)는 코로나1...,아성다이소,박정부,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,#^DAT^1967#년 프로 야구 드래프트 1순위로 @*ORG*요미우리 자이언츠@에...,요미우리 자이언츠,1967,ORG,DAT,no_relation,"[22, 30]","[0, 3]"
...,...,...,...,...,...,...,...,...,...
32423,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,유기준,부산 서구·동구,PER,LOC,per:employee_of,"[93, 95]","[100, 107]"
32424,32466,"법포는 다시 @*PER*최시형@, 서병학, #^PER^손병희# 직계인 북접과 다시 ...",최시형,손병희,PER,PER,per:colleagues,"[7, 9]","[17, 19]"
32425,32467,@*ORG*완도군@(군수 #^PER^신우철#)이 국토교통부에서 실시한 '2019 교...,완도군,신우철,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
32426,32468,"중앙일보, @*ORG*JTBC@ 회장을 지낸 이후 #^ORG^중앙홀딩스# 회장, 재...",JTBC,중앙홀딩스,ORG,ORG,no_relation,"[6, 9]","[21, 25]"


In [185]:
update_dataset = update_idx(train_dataset)
# 30% 확률로 swap해주는 기능
dataset = random_swap(update_dataset, 0.3)

In [186]:
swap_30percent = augmentation(train_dataset)

In [187]:
# 총 swap된 sentence를 count
count = 0

# swap된 sentence와 original sentence를 비교해 어떤 단어가 바뀌었는지 알려준다.
for original_sentence, swap_sentence in zip(train_dataset['sentence'], swap_30percent['sentence']):
    
    # 교환된 단어 찾기
    original_words = original_sentence.split()
    swap_words = swap_sentence.split()
    
    # swap문장과 기존 문장을 비교해 교환된 단어를 알려준다.
    for original_word, swap_word in zip(original_words, swap_words):
        if original_word != swap_word:
            count += 1
            print("#",count)
            print("Original Sentence:", original_sentence)
            print("Swap Sentence:", swap_sentence)
            print("Swapped Word Pair:", original_word, " <-> ", swap_word)
            print('=' * 90)
            break
    
    # 예시 20개만 확인하기
    if count == 20:
        break

# 1
Original Sentence: 〈Something〉는 #^PER^조지 해리슨#이 쓰고 @*ORG*비틀즈@가 1969년 앨범 《Abbey Road》에 담은 노래다.
Swap Sentence: 〈Something〉는 #^PER^조지 해리슨#이 쓰고 @*ORG*비틀즈@가 1969년 앨범 담은 Road》에 《Abbey 노래다.
Swapped Word Pair: 《Abbey  <->  담은
# 2
Original Sentence: #^DAT^1967#년 프로 야구 드래프트 1순위로 @*ORG*요미우리 자이언츠@에게 입단하면서 등번호는 8번으로 배정되었다.
Swap Sentence: #^DAT^1967#년 프로 8번으로 드래프트 1순위로 @*ORG*요미우리 자이언츠@에게 입단하면서 등번호는 야구 배정되었다.
Swapped Word Pair: 야구  <->  8번으로
# 3
Original Sentence: 이른바 'Z세대'로 불리는 1990년대 중반 이후 태어난 세대에게 대표 아이콘으로 통하는 미국 #^POH^싱어송라이터# @*PER*빌리 아일리시@(본명 빌리 오코널, 19)가 팝 역사를 새로 썼다.
Swap Sentence: 이른바 'Z세대'로 불리는 1990년대 중반 이후 태어난 세대에게 대표 미국 통하는 아이콘으로 #^POH^싱어송라이터# @*PER*빌리 아일리시@(본명 빌리 오코널, 19)가 팝 역사를 새로 썼다.
Swapped Word Pair: 아이콘으로  <->  미국
# 4
Original Sentence: 2009년 9월, 미국 프로 야구 @*ORG*필라델피아 필리스@ 소속의 야구 선수 #^PER^박찬호#는 《MBC 스페셜-박찬호는 당신을 잊지 않았다》 편에서 “최진실 씨의 아픔과 죽음의 고통을 이해합니다. 최진실 씨 사건에 눈물을 흘렸습니다. 저도 죽으려고 마음을 먹었던 적이 있었습니다. 잘하려고 애를 쓰는데 비난과 비판이 쏟아졌습니다. 머리가 빠지고 너무 힘들었습니다”라고 말하며 최진실의 죽음에 대해 안타까움을 표현했다.
Swap Sente

In [188]:
# 30% 확률로 delete해주는 기능
dataset = random_swap(update_dataset, 0.3)

In [ ]:
''' 현재 시간이 많이 소요되어서 작성하지 못함. 추후에 적을 예정 
    아직 코드 돌리지 말 것!'''
delete_30percent = augmentation(train_dataset)
delete_30percent